In [29]:
import numpy as np
import cv2
import operator
import numpy as np
from matplotlib import pyplot as plt
IMAGE_URL = "./sudoku_1.jpg"
SUDOKU_SIZE = 9

In [30]:
def display_image(img):
    cv2.imshow('Sudoku', img)
    cv2.waitKey(0)  # Wait for any key to be pressed (with the image window active)
    cv2.destroyAllWindows()  # Close all windows
    return img

In [31]:
def preprocess(img, skip_dilate=False):
    """Uses a blurring function, adaptive thresholding and dilation to expose the main features of an image."""
    # Gaussian blur with a kernal size (height, width) of 9.
    # Note that kernal sizes must be positive and odd and the kernel must be square.
    #https://www.tutorialspoint.com/opencv/opencv_gaussian_blur.htm
    blur_img = cv2.GaussianBlur(img.copy(), (9, 9), 0)
    display_image(blur_img)
	# Adaptive threshold using 11 nearest neighbour pixels
    # https://www.pyimagesearch.com/2021/05/12/adaptive-thresholding-with-opencv-cv2-adaptivethreshold/
    # https://www.tutorialspoint.com/opencv/opencv_adaptive_threshold.htm
    threshold_img = cv2.adaptiveThreshold(blur_img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    #display_image(threshold_img)
    # Invert colours, so gridlines have non-zero pixel values.
	# Necessary to dilate the image, otherwise will look like erosion instead.
    processed_img = cv2.bitwise_not(threshold_img, threshold_img)
    #display_image(processed_img)

    if not skip_dilate:
    	# Dilate the image to increase the size of the grid lines.
    	kernel = np.array([[0., 1., 0.], [1., 1., 1.], [0., 1., 0.]],np.uint8)
    	processed_img = cv2.dilate(processed_img, kernel)

    return processed_img

In [32]:
def find_corners_of_largest_polygon(img):
	"""Finds the 4 extreme corners of the largest contour in the image."""
	#using copy of img as findcontours alters the original image
	# https://docs.opencv.org/master/d9/d8b/tutorial_py_contours_hierarchy.html
	"""The CHAIN_APPROX_SIMPLE  algorithm compresses horizontal, vertical, and diagonal segments along the 
	contour and leaves only their end points. This means that any of the points along the straight paths 
	will be dismissed, and we will be left with only the end points. For example, consider a contour, 
	along a rectangle. 
	All the contour points, except the four corner points will be dismissed. 
	"""
	contours, hierarchy = cv2.findContours(img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # Find contours
	contours = sorted(contours, key = cv2.contourArea, reverse = True)  # Sort by area, descending
	largest_polygon = contours[0]  # Largest image

	# Use of `operator.itemgetter` with `max` and `min` allows us to get the index of the point
	# Each point is an array of 1 coordinate, hence the [0] getter, then [0] or [1] used to get x and y respectively.

	# Bottom-right point has the largest (x + y) value
	# Top-left has point smallest (x + y) value
	# Bottom-left point has smallest (x - y) value
	# Top-right point has largest (x - y) value
	bottom_right = max(enumerate([point[0][0] + point[0][1] for point in largest_polygon]), key = operator.itemgetter(1))[0]
	top_left = min(enumerate([point[0][0] + point[0][1] for point in largest_polygon]), key = operator.itemgetter(1))[0]
	bottom_left = min(enumerate([point[0][0] - point[0][1] for point in largest_polygon]), key = operator.itemgetter(1))[0]
	top_right = max(enumerate([point[0][0] - point[0][1] for point in largest_polygon]), key = operator.itemgetter(1))[0]

	# Return an array of all 4 points using the indices
	# Each point is in its own array of one coordinate
	#Order: TL, TR, BR, BL (CLOCKWISE)
	return [largest_polygon[top_left][0], largest_polygon[top_right][0], largest_polygon[bottom_right][0], largest_polygon[bottom_left][0]]


In [33]:
def distance(point1, point2):
	"""Returns the scalar distance between two points"""
	a = point2[0] - point1[0]
	b = point2[1] - point1[1]
	return np.sqrt((a ** 2) + (b ** 2))

In [34]:
def crop_and_warp(img, corners):
	"""Crops and warps a rectangular section from an image into a square of similar size."""

	# Rectangle described by top left, top right, bottom right and bottom left points
	top_left, top_right, bottom_right, bottom_left = corners[0], corners[1], corners[2], corners[3]

	# Explicitly set the data type to float32 or `getPerspectiveTransform` will throw an error
	img_float32 = np.array([top_left, top_right, bottom_right, bottom_left], dtype='float32')

	# Get the longest side in the rectangle
	side_length = max([
		distance(bottom_right, top_right),
		distance(top_left, bottom_left),
		distance(bottom_right, bottom_left),
		distance(top_left, top_right)
	])
	#https://theailearner.com/tag/cv2-getperspectivetransform/
	#https://www.pyimagesearch.com/2014/08/25/4-point-opencv-getperspective-transform-example/
	# Describe a square with side of the calculated length, this is the new perspective we want to warp to
	new_perspective = np.array([[0, 0], [side_length - 1, 0], [side_length - 1, side_length - 1], [0, side_length - 1]], dtype='float32')

	# Gets the transformation matrix for skewing the image to fit a square by comparing the 4 before and after points
	transformed_img = cv2.getPerspectiveTransform(img_float32, new_perspective)
	# Performs the transformation on the original image
	return cv2.warpPerspective(img, transformed_img, (int(side_length), int(side_length)))


In [35]:
def all_cell_coordinates(img):
	"""Infers 81 cell grid from a square image."""
	squares = []
	side = img.shape[:1]
	side = side[0] / SUDOKU_SIZE

	# Note that we swap j and i here so the rectangles are stored in the list reading left-right instead of top-down.
	for j in range(SUDOKU_SIZE):
		for i in range(SUDOKU_SIZE):
			p1 = (i * side, j * side)  # Top left corner of a bounding box
			p2 = ((i + 1) * side, (j + 1) * side)  # Bottom right corner of bounding box
			squares.append((p1, p2))
	return squares

In [36]:
def get_digit_cell(img, cell):
	"""Cuts a rectangle from an image using the top left and bottom right points."""
	return img[int(cell[0][1]):int(cell[1][1]), int(cell[0][0]):int(cell[1][0])]


In [37]:
def find_largest_feature(inp_img, scan_tl=None, scan_br=None):
    """
    Uses the fact the `floodFill` function returns a bounding box of the area it filled to find the biggest
    connected pixel structure in the image. Fills this structure in white, reducing the rest to black.
    """
    img = inp_img.copy()  # Copy the image, leaving the original untouched
    height, width = img.shape[:2]

    max_area = 0
    seed_point = (None, None)

    if scan_tl is None:
        scan_tl = [0, 0]

    if scan_br is None:
        scan_br = [width, height]

    # Loop through the image
    for x in range(scan_tl[0], scan_br[0]):
        for y in range(scan_tl[1], scan_br[1]):
            # Only operate on light or white squares
            # Note that .item() appears to take input as y, x
            if img.item(y, x) == 255 and x < width and y < height:
                area = cv2.floodFill(img, None, (x, y), 64)
                if area[0] > max_area:  # Gets the maximum bound area which should be the grid
                    max_area = area[0]
                    seed_point = (x, y)

    # Colour everything grey (compensates for features outside of our middle scanning range
    for x in range(width):
        for y in range(height):
            if img.item(y, x) == 255 and x < width and y < height:
                cv2.floodFill(img, None, (x, y), 64)

    # Mask that is 2 pixels bigger than the image
    mask = np.zeros((height + 2, width + 2), np.uint8)

    # Highlight the main feature
    if all([p is not None for p in seed_point]):
        cv2.floodFill(img, mask, seed_point, 255)

    top, bottom, left, right = height, 0, width, 0

    for x in range(width):
        for y in range(height):
            if img.item(y, x) == 64:  # Hide anything that isn't the main feature
                cv2.floodFill(img, mask, (x, y), 0)

            # Find the bounding parameters
            if img.item(y, x) == 255:
                top = y if y < top else top
                bottom = y if y > bottom else bottom
                left = x if x < left else left
                right = x if x > right else right

    bbox = [[left, top], [right, bottom]]
    return img, np.array(bbox, dtype='float32'), seed_point

In [38]:
def scale_and_centre(img, size, margin=0, background=0):
    """Scales and centres an image onto a new background square."""
    h, w = img.shape[:2]

    def centre_pad(length):
        """Handles centering for a given length that may be odd or even."""
        if length % 2 == 0:
            side1 = int((size - length) / 2)
            side2 = side1
        else:
            side1 = int((size - length) / 2)
            side2 = side1 + 1
        return side1, side2

    def scale(r, x):
        return int(r * x)

    if h > w:
        t_pad = int(margin / 2)
        b_pad = t_pad
        ratio = (size - margin) / h
        w, h = scale(ratio, w), scale(ratio, h)
        l_pad, r_pad = centre_pad(w)
    else:
        l_pad = int(margin / 2)
        r_pad = l_pad
        ratio = (size - margin) / w
        w, h = scale(ratio, w), scale(ratio, h)
        t_pad, b_pad = centre_pad(h)

    img = cv2.resize(img, (w, h))
    img = cv2.copyMakeBorder(img, t_pad, b_pad, l_pad,
                             r_pad, cv2.BORDER_CONSTANT, None, background)
    return cv2.resize(img, (size, size))


In [39]:
def extract_digit(img, cell, size):
	"""Extracts a digit (if one exists) from a Sudoku square."""

	digit = get_digit_cell(img, cell)  # Get the digit box from the whole square

	# Use fill feature finding to get the largest feature in middle of the box
	# Margin used to define an area in the middle we would expect to find a pixel belonging to the digit
	height, width = digit.shape[:2]
	margin = int(np.mean([height, width]) / 2.5) ##NO IDEA WHY DIV BY 2.5
	_, bbox, seed = find_largest_feature(digit, [margin, margin], [width - margin, height - margin])
	digit = get_digit_cell(digit, bbox)

	# Scale and pad the digit so that it fits a square of the digit size we're using for machine learning
	width = bbox[1][0] - bbox[0][0]
	height = bbox[1][1] - bbox[0][1]

	# Ignore any small bounding boxes
	if width > 0 and height > 0 and (width * height) > 100 and len(digit) > 0:
		return scale_and_centre(digit, size, 4)
	else:
		return np.zeros((size, size), np.uint8)

In [40]:
def get_digits(img, cell_coordinates, size):
    """Extracts digits from their cells and builds an array"""
    digits = []
    img = preprocess(img.copy(), skip_dilate=True)
    #cv2.imshow('img', img)
    for cell in cell_coordinates:
        digits.append(extract_digit(img, cell, size))
    return digits

In [41]:
def show_digits(digits, colour=255):
    """Shows list of 81 extracted digits in a grid format"""
    grid = []
    with_border = [cv2.copyMakeBorder(img.copy(), 1, 1, 1, 1, cv2.BORDER_CONSTANT, None, colour) for img in digits]
    for i in range(SUDOKU_SIZE):
        row = np.concatenate(with_border[i * SUDOKU_SIZE:((i + 1) * SUDOKU_SIZE)], axis=1)
        grid.append(row)
    img = display_image(np.concatenate(grid))
    return img

In [43]:
def extract_sudoku(path):
    original_img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    display_image(original_img)
    processed_img = preprocess(original_img)
    corners = find_corners_of_largest_polygon(processed_img)
    #print(corners)
    cropped = crop_and_warp(original_img, corners)
    #display_image(cropped)    
    cell_coordinates = all_cell_coordinates(cropped)
    print(cell_coordinates)
    # digits = get_digits(cropped, cell_coordinates, 28)
    # print(digits)
    # final_image = show_digits(digits)
    # return final_image
extract_sudoku(IMAGE_URL)

[((0.0, 0.0), (38.111111111111114, 38.111111111111114)), ((38.111111111111114, 0.0), (76.22222222222223, 38.111111111111114)), ((76.22222222222223, 0.0), (114.33333333333334, 38.111111111111114)), ((114.33333333333334, 0.0), (152.44444444444446, 38.111111111111114)), ((152.44444444444446, 0.0), (190.55555555555557, 38.111111111111114)), ((190.55555555555557, 0.0), (228.66666666666669, 38.111111111111114)), ((228.66666666666669, 0.0), (266.7777777777778, 38.111111111111114)), ((266.7777777777778, 0.0), (304.8888888888889, 38.111111111111114)), ((304.8888888888889, 0.0), (343.0, 38.111111111111114)), ((0.0, 38.111111111111114), (38.111111111111114, 76.22222222222223)), ((38.111111111111114, 38.111111111111114), (76.22222222222223, 76.22222222222223)), ((76.22222222222223, 38.111111111111114), (114.33333333333334, 76.22222222222223)), ((114.33333333333334, 38.111111111111114), (152.44444444444446, 76.22222222222223)), ((152.44444444444446, 38.111111111111114), (190.55555555555557, 76.2222